# Web Scraping: Thailand Yellow Pages

## Objective
ดึงข้อมูลรายชื่อธุรกิจประเภท **คลินิก** จากเว็บไซต์ Thailand Yellow Pages (https://www.yellowpages.co.th/) และจัดเก็บใน Pandas DataFrame

## Data to Extract
- ชื่อธุรกิจ (Business Name)
- ที่อยู่/จังหวัด (Location/Province)
- เบอร์โทรศัพท์ (Phone Number)
- รายละเอียดบริการ (Description)
- เว็บไซต์ (Website - if available)

## Approach
1. ใช้ `requests` เพื่อดึง HTML จากเว็บไซต์
2. ใช้ `BeautifulSoup` เพื่อ parse HTML และดึงข้อมูล
3. จัดเก็บข้อมูลใน Pandas DataFrame
4. Export เป็น CSV file

## 1. Import Libraries

In [1]:
# Install required packages (uncomment if needed)
# !pip install requests beautifulsoup4 pandas lxml

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Configuration

กำหนดค่า URL และ Headers สำหรับการ request

In [2]:
# Base URL for clinic listings
BASE_URL = "https://www.yellowpages.co.th"
CATEGORY_URL = f"{BASE_URL}/heading/คลินิก"

# Headers to mimic browser request
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'th-TH,th;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

# Number of pages to scrape (limit for demo)
MAX_PAGES = 5

print(f"Target URL: {CATEGORY_URL}")
print(f"Pages to scrape: {MAX_PAGES}")

Target URL: https://www.yellowpages.co.th/heading/คลินิก
Pages to scrape: 5


## 3. Helper Functions

สร้างฟังก์ชันสำหรับการดึงและ parse ข้อมูล

In [4]:
def fetch_page(url: str, retries: int = 3) -> Optional[BeautifulSoup]:
    """
    Fetch HTML content from URL and return BeautifulSoup object.
    
    Args:
        url: URL to fetch
        retries: Number of retry attempts
    
    Returns:
        BeautifulSoup object or None if failed
    """
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            return BeautifulSoup(response.content, 'lxml')
        except requests.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(2)
    return None


def extract_text(element, default: str = "") -> str:
    """
    Safely extract text from BeautifulSoup element.
    
    Args:
        element: BeautifulSoup element
        default: Default value if element is None
    
    Returns:
        Extracted text or default value
    """
    if element:
        return element.get_text(strip=True)
    return default


print("Helper functions defined!")

Helper functions defined!


## 4. Analyze Page Structure

ดึงหน้าแรกมาวิเคราะห์โครงสร้าง HTML

In [5]:
# Fetch the first page to analyze structure
soup = fetch_page(CATEGORY_URL)

if soup:
    print("Page fetched successfully!")
    print(f"Page title: {soup.title.string if soup.title else 'N/A'}")
else:
    print("Failed to fetch page")

Page fetched successfully!
Page title: คลินิก | หมวดหมู่ | ไทยแลนด์ เยลโล่เพจเจส


In [6]:
# Analyze page structure - find business listing containers
if soup:
    # Use the correct selector for Yellow Pages
    listings = soup.select('div.yp-search-listing')
    print(f"Found {len(listings)} business listings on the page")
    
    if listings:
        # Show first listing structure
        first_listing = listings[0]
        print("\n--- First listing preview ---")
        
        # Extract sample data
        name = first_listing.select_one('.yp-listing-title h3 a')
        address = first_listing.select_one('p.yp-listing-address')
        desc = first_listing.select_one('p.yp-listing-desc')
        
        if name:
            print(f"Name: {name.get_text(strip=True)}")
        if address:
            print(f"Address: {address.get_text(strip=True)}")
        if desc:
            print(f"Description: {desc.get_text(strip=True)[:100]}...")
else:
    print("Failed to fetch page - check connection")

Found 10 business listings on the page

--- First listing preview ---
Name: คลินิก บางบัวทอง - ภัทร บางบัวทองคลินิกเวชกรรม
Address: ตำบลบางบัวทอง อำเภอบางบัวทอง จังหวัดนนทบุรี 11110
Description: ตรวจรักษาโรคทั่วไป หัตถการ พ่นยา ทำแผล สำหรับเด็ก ผู้ใหญ่ หญิงตั้งครรภ์ ผู้สูงอายุ

ตอบโจทย์การค้นหา...


## 5. Scraping Function

ฟังก์ชันหลักสำหรับดึงข้อมูลธุรกิจจากแต่ละหน้า

In [7]:
def parse_business_listing(card) -> Dict:
    """
    Parse a single business listing card and extract information.

    Structure of Yellow Pages listing (based on actual HTML analysis):
    - Container: div.yp-search-listing
    - Name: h3 a (inside yp-listing-title)
    - Website: a.yp-listing-website
    - Address: p.yp-listing-address
    - Description: p.yp-listing-desc
    - Category: listing-category-section a

    Args:
        card: BeautifulSoup element containing business info

    Returns:
        Dictionary with business information
    """
    business = {
        'name': '',
        'address': '',
        'description': '',
        'website': '',
        'category': '',
        'profile_url': ''
    }

    try:
        # Extract business name from h3 > a inside yp-listing-title
        name_elem = card.select_one('.yp-listing-title h3 a')
        if name_elem:
            business['name'] = extract_text(name_elem)
            business['profile_url'] = name_elem.get('href', '')

        # Extract address from p.yp-listing-address
        address_elem = card.select_one('p.yp-listing-address')
        if address_elem:
            business['address'] = extract_text(address_elem)

        # Extract description from p.yp-listing-desc
        desc_elem = card.select_one('p.yp-listing-desc')
        if desc_elem:
            # Clean description and limit length
            desc_text = extract_text(desc_elem)
            business['description'] = desc_text[:300] if len(desc_text) > 300 else desc_text

        # Extract website from a.yp-listing-website
        website_elem = card.select_one('a.yp-listing-website')
        if website_elem:
            business['website'] = website_elem.get('href', '')

        # Extract category from listing-category-section
        category_elem = card.select_one('.listing-category-section a')
        if category_elem:
            business['category'] = extract_text(category_elem)

    except Exception as e:
        print(f"Error parsing listing: {e}")

    return business


def scrape_page(url: str) -> List[Dict]:
    """
    Scrape all business listings from a single page.

    Args:
        url: Page URL to scrape

    Returns:
        List of business dictionaries
    """
    businesses = []
    soup = fetch_page(url)

    if not soup:
        return businesses

    # Find all business listing cards using the correct selector
    # Each listing is in: div.yp-search-listing
    cards = soup.select('div.yp-search-listing')

    print(f"  Found {len(cards)} listing cards on page")

    for card in cards:
        business = parse_business_listing(card)
        if business['name']:  # Only add if name was found
            businesses.append(business)

    return businesses


print("Scraping functions defined!")

Scraping functions defined!


## 6. Execute Scraping

ดึงข้อมูลจากหลายหน้า

In [8]:
def scrape_all_pages(base_url: str, max_pages: int) -> List[Dict]:
    """
    Scrape multiple pages of business listings.
    
    Args:
        base_url: Base category URL
        max_pages: Maximum number of pages to scrape
    
    Returns:
        List of all business dictionaries
    """
    all_businesses = []
    
    for page in range(1, max_pages + 1):
        if page == 1:
            url = base_url
        else:
            url = f"{base_url}/page/{page}"
        
        print(f"Scraping page {page}/{max_pages}: {url}")
        
        businesses = scrape_page(url)
        all_businesses.extend(businesses)
        
        print(f"  Found {len(businesses)} businesses (Total: {len(all_businesses)})")
        
        # Polite delay between requests
        if page < max_pages:
            delay = random.uniform(1, 3)
            print(f"  Waiting {delay:.1f}s...")
            time.sleep(delay)
    
    return all_businesses


# Execute scraping
print("Starting scraping...\n")
all_businesses = scrape_all_pages(CATEGORY_URL, MAX_PAGES)
print(f"\nScraping complete! Total businesses collected: {len(all_businesses)}")

Starting scraping...

Scraping page 1/5: https://www.yellowpages.co.th/heading/คลินิก
  Found 10 listing cards on page
  Found 10 businesses (Total: 10)
  Waiting 2.7s...
Scraping page 2/5: https://www.yellowpages.co.th/heading/คลินิก/page/2
Attempt 1 failed: 404 Client Error: Not Found for url: https://www.yellowpages.co.th/heading/%E0%B8%84%E0%B8%A5%E0%B8%B4%E0%B8%99%E0%B8%B4%E0%B8%81/page/2
Attempt 2 failed: 404 Client Error: Not Found for url: https://www.yellowpages.co.th/heading/%E0%B8%84%E0%B8%A5%E0%B8%B4%E0%B8%99%E0%B8%B4%E0%B8%81/page/2
Attempt 3 failed: 404 Client Error: Not Found for url: https://www.yellowpages.co.th/heading/%E0%B8%84%E0%B8%A5%E0%B8%B4%E0%B8%99%E0%B8%B4%E0%B8%81/page/2
  Found 0 businesses (Total: 10)
  Waiting 1.8s...
Scraping page 3/5: https://www.yellowpages.co.th/heading/คลินิก/page/3
Attempt 1 failed: 404 Client Error: Not Found for url: https://www.yellowpages.co.th/heading/%E0%B8%84%E0%B8%A5%E0%B8%B4%E0%B8%99%E0%B8%B4%E0%B8%81/page/3
Attempt 2 failed

## 7. Create DataFrame

แปลงข้อมูลที่ดึงมาเป็น Pandas DataFrame

In [9]:
# Create DataFrame from scraped data
df = pd.DataFrame(all_businesses)

# Display basic info
print(f"DataFrame shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")

DataFrame shape: (10, 6)

Columns: ['name', 'address', 'description', 'website', 'category', 'profile_url']

Data types:
name           object
address        object
description    object
website        object
category       object
profile_url    object
dtype: object


In [10]:
# Display first few rows
df.head(10)

,name,address,description,website,category,profile_url
0,คลินิก บางบัวทอง - ภัทร บางบัวทองคลินิกเวชกรรม,ตำบลบางบัวทอง อำเภอบางบัวทอง จังหวัดนนทบุรี 11110,ตรวจรักษาโรคทั่วไป หัตถการ พ่นยา ทำแผล สำหรับเ...,,คลินิก,https://www.yellowpages.co.th/profile/%E0%B8%8...
1,คลินิกรักษาสัตว์ ยุทธชาติรักษ์สัตว์ พัทลุง,ตำบลคูหาสวรรค์ อำเภอเมืองพัทลุง จังหวัดพัทลุง ...,ยุทธชาติรักษาสัตว์คลินิกรักษาสัตว์เลี้ยง สัตว์...,https://yutthachatruksasat.yellowpages.co.th,โรงพยาบาลและคลินิกสัตวแพทย์,https://yutthachatruksasat.yellowpages.co.th
2,หมอรัก(ษ์)หมา,ตำบลนครสวรรค์ตก อำเภอเมืองนครสวรรค์ จังหวัดนคร...,คลินิกรักษาสัตว์เลี้ยง\n\n\n\nหากน้องหมาและแมว...,https://mohrakma.yellowpages.co.th,โรงพยาบาลและคลินิกสัตวแพทย์,https://mohrakma.yellowpages.co.th
3,พระยาสุเรนทร์คลินิกเวชกรรมสาขาแกรนด์ออคิด,แขวงบางชัน เขตคลองสามวา กรุงเทพมหานคร 10510,พระยาสุเรนทร์คลินิกเวชกรรม สาขาแกรนด์ออคิด ให้...,,คลินิก,https://www.yellowpages.co.th/profile/%E0%B8%9...
4,โรงพยาบาล และคลินิกสัตว์ สระบุรี,ตำบลปากเพรียว อำเภอเมืองสระบุรี จังหวัดสระบุรี...,โรงพยาบาลสัตว์เมืองเพรียว สระบุรี \n\nดูแลรักษ...,,โรงพยาบาลและคลินิกสัตวแพทย์,https://www.yellowpages.co.th/profile/%E0%B9%8...
5,พงศ์ดนัยคลินิกกระดูกและข้อ โคราช นครราชสีมา,ตำบลในเมือง อำเภอเมืองนครราชสีมา จังหวัดนครราช...,สอบถาม-นัดหมาย พงศ์ดนัยคลินิกกระดูกและข้อ โทร ...,,คลินิก,https://www.yellowpages.co.th/profile/%E0%B8%9...
6,คมวิชช์ศัลยกรรมตกแต่ง,แขวงอรุณอมรินทร์ เขตบางกอกน้อย กรุงเทพมหานคร 1...,แพทย์ประจำคลินิกศัลยกรรมตกแต่ง โรงพยาบาลบำรุงร...,,คลินิก,https://www.yellowpages.co.th/profile/%E0%B8%8...
7,พระยาสุเรนทร์คลินิกเวชกรรมสาขาแกรนด์ออคิด,แขวงบางชัน เขตคลองสามวา กรุงเทพมหานคร 10510,พระยาสุเรนทร์คลินิกเวชกรรม สาขาแกรนด์ออคิด ให้...,,คลินิก,https://www.yellowpages.co.th/profile/%E0%B8%9...
8,คลินิกศัลยกรรมตกแต่งเสริมสวย รังสิต,ตำบลประชาธิปัตย์ อำเภอธัญบุรี จังหวัดปทุมธานี ...,"ศัลยกรรมตกแต่งเสริมสวย,คลินิกศัลยกรรมตกแต่งเสร...",,คลินิก,https://www.yellowpages.co.th/profile/%E0%B8%8...
9,พระยาสุเรนทร์คลินิกเวชกรรมสาขาแกรนด์ออคิด,แขวงบางชัน เขตคลองสามวา กรุงเทพมหานคร 10510,คลินิก. ลองสามวา\n\t\t\t22.คลินิก. รามอินทรา\n...,,คลินิก,https://www.yellowpages.co.th/profile/%E0%B8%9...


In [11]:
# Data summary statistics
print("=" * 50)
print("DATA SUMMARY")
print("=" * 50)
print(f"\nTotal records: {len(df)}")
print(f"\nNon-empty value counts:")
for col in df.columns:
    non_empty = (df[col] != '').sum()
    pct = (non_empty/len(df)*100) if len(df) > 0 else 0
    print(f"  {col}: {non_empty} ({pct:.1f}%)")

DATA SUMMARY

Total records: 10

Non-empty value counts:
  name: 10 (100.0%)
  address: 10 (100.0%)
  description: 10 (100.0%)
  website: 2 (20.0%)
  category: 10 (100.0%)
  profile_url: 10 (100.0%)


## 8. Data Cleaning

ทำความสะอาดข้อมูลที่ดึงมา

In [12]:
# Clean the data
df_cleaned = df.copy()

# Remove duplicates based on name
initial_count = len(df_cleaned)
df_cleaned = df_cleaned.drop_duplicates(subset=['name'], keep='first')
print(f"Removed {initial_count - len(df_cleaned)} duplicate entries")

# Remove rows with empty names
df_cleaned = df_cleaned[df_cleaned['name'] != '']
print(f"Records after cleaning: {len(df_cleaned)}")

# Reset index
df_cleaned = df_cleaned.reset_index(drop=True)

# Reorder columns for better readability
column_order = ['name', 'category', 'address', 'description', 'website', 'profile_url']
df_cleaned = df_cleaned[column_order]

df_cleaned.head(10)

Removed 2 duplicate entries
Records after cleaning: 8


,name,category,address,description,website,profile_url
0,คลินิก บางบัวทอง - ภัทร บางบัวทองคลินิกเวชกรรม,คลินิก,ตำบลบางบัวทอง อำเภอบางบัวทอง จังหวัดนนทบุรี 11110,ตรวจรักษาโรคทั่วไป หัตถการ พ่นยา ทำแผล สำหรับเ...,,https://www.yellowpages.co.th/profile/%E0%B8%8...
1,คลินิกรักษาสัตว์ ยุทธชาติรักษ์สัตว์ พัทลุง,โรงพยาบาลและคลินิกสัตวแพทย์,ตำบลคูหาสวรรค์ อำเภอเมืองพัทลุง จังหวัดพัทลุง ...,ยุทธชาติรักษาสัตว์คลินิกรักษาสัตว์เลี้ยง สัตว์...,https://yutthachatruksasat.yellowpages.co.th,https://yutthachatruksasat.yellowpages.co.th
2,หมอรัก(ษ์)หมา,โรงพยาบาลและคลินิกสัตวแพทย์,ตำบลนครสวรรค์ตก อำเภอเมืองนครสวรรค์ จังหวัดนคร...,คลินิกรักษาสัตว์เลี้ยง\n\n\n\nหากน้องหมาและแมว...,https://mohrakma.yellowpages.co.th,https://mohrakma.yellowpages.co.th
3,พระยาสุเรนทร์คลินิกเวชกรรมสาขาแกรนด์ออคิด,คลินิก,แขวงบางชัน เขตคลองสามวา กรุงเทพมหานคร 10510,พระยาสุเรนทร์คลินิกเวชกรรม สาขาแกรนด์ออคิด ให้...,,https://www.yellowpages.co.th/profile/%E0%B8%9...
4,โรงพยาบาล และคลินิกสัตว์ สระบุรี,โรงพยาบาลและคลินิกสัตวแพทย์,ตำบลปากเพรียว อำเภอเมืองสระบุรี จังหวัดสระบุรี...,โรงพยาบาลสัตว์เมืองเพรียว สระบุรี \n\nดูแลรักษ...,,https://www.yellowpages.co.th/profile/%E0%B9%8...
5,พงศ์ดนัยคลินิกกระดูกและข้อ โคราช นครราชสีมา,คลินิก,ตำบลในเมือง อำเภอเมืองนครราชสีมา จังหวัดนครราช...,สอบถาม-นัดหมาย พงศ์ดนัยคลินิกกระดูกและข้อ โทร ...,,https://www.yellowpages.co.th/profile/%E0%B8%9...
6,คมวิชช์ศัลยกรรมตกแต่ง,คลินิก,แขวงอรุณอมรินทร์ เขตบางกอกน้อย กรุงเทพมหานคร 1...,แพทย์ประจำคลินิกศัลยกรรมตกแต่ง โรงพยาบาลบำรุงร...,,https://www.yellowpages.co.th/profile/%E0%B8%8...
7,คลินิกศัลยกรรมตกแต่งเสริมสวย รังสิต,คลินิก,ตำบลประชาธิปัตย์ อำเภอธัญบุรี จังหวัดปทุมธานี ...,"ศัลยกรรมตกแต่งเสริมสวย,คลินิกศัลยกรรมตกแต่งเสร...",,https://www.yellowpages.co.th/profile/%E0%B8%8...


## 9. Export to CSV

บันทึกข้อมูลเป็นไฟล์ CSV

In [13]:
# Export to CSV
output_file = 'clinic_listings_yellowpages.csv'
df_cleaned.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Data exported to: {output_file}")
print(f"Total records: {len(df_cleaned)}")

Data exported to: clinic_listings_yellowpages.csv
Total records: 8


## 10. Summary

สรุปผลการดึงข้อมูล

In [14]:
print("=" * 60)
print("WEB SCRAPING SUMMARY")
print("=" * 60)
print(f"\nSource: Thailand Yellow Pages (yellowpages.co.th)")
print(f"Category: คลินิก (Clinic)")
print(f"Pages scraped: {MAX_PAGES}")
print(f"Total records collected: {len(df_cleaned)}")
print(f"Output file: {output_file}")
print("\nColumns in dataset:")
for col in df_cleaned.columns:
    print(f"  - {col}")
print("\n" + "=" * 60)

WEB SCRAPING SUMMARY

Source: Thailand Yellow Pages (yellowpages.co.th)
Category: คลินิก (Clinic)
Pages scraped: 5
Total records collected: 8
Output file: clinic_listings_yellowpages.csv

Columns in dataset:
  - name
  - category
  - address
  - description
  - website
  - profile_url



In [15]:
# Final display of data
df_cleaned

,name,category,address,description,website,profile_url
0,คลินิก บางบัวทอง - ภัทร บางบัวทองคลินิกเวชกรรม,คลินิก,ตำบลบางบัวทอง อำเภอบางบัวทอง จังหวัดนนทบุรี 11110,ตรวจรักษาโรคทั่วไป หัตถการ พ่นยา ทำแผล สำหรับเ...,,https://www.yellowpages.co.th/profile/%E0%B8%8...
1,คลินิกรักษาสัตว์ ยุทธชาติรักษ์สัตว์ พัทลุง,โรงพยาบาลและคลินิกสัตวแพทย์,ตำบลคูหาสวรรค์ อำเภอเมืองพัทลุง จังหวัดพัทลุง ...,ยุทธชาติรักษาสัตว์คลินิกรักษาสัตว์เลี้ยง สัตว์...,https://yutthachatruksasat.yellowpages.co.th,https://yutthachatruksasat.yellowpages.co.th
2,หมอรัก(ษ์)หมา,โรงพยาบาลและคลินิกสัตวแพทย์,ตำบลนครสวรรค์ตก อำเภอเมืองนครสวรรค์ จังหวัดนคร...,คลินิกรักษาสัตว์เลี้ยง\n\n\n\nหากน้องหมาและแมว...,https://mohrakma.yellowpages.co.th,https://mohrakma.yellowpages.co.th
3,พระยาสุเรนทร์คลินิกเวชกรรมสาขาแกรนด์ออคิด,คลินิก,แขวงบางชัน เขตคลองสามวา กรุงเทพมหานคร 10510,พระยาสุเรนทร์คลินิกเวชกรรม สาขาแกรนด์ออคิด ให้...,,https://www.yellowpages.co.th/profile/%E0%B8%9...
4,โรงพยาบาล และคลินิกสัตว์ สระบุรี,โรงพยาบาลและคลินิกสัตวแพทย์,ตำบลปากเพรียว อำเภอเมืองสระบุรี จังหวัดสระบุรี...,โรงพยาบาลสัตว์เมืองเพรียว สระบุรี \n\nดูแลรักษ...,,https://www.yellowpages.co.th/profile/%E0%B9%8...
5,พงศ์ดนัยคลินิกกระดูกและข้อ โคราช นครราชสีมา,คลินิก,ตำบลในเมือง อำเภอเมืองนครราชสีมา จังหวัดนครราช...,สอบถาม-นัดหมาย พงศ์ดนัยคลินิกกระดูกและข้อ โทร ...,,https://www.yellowpages.co.th/profile/%E0%B8%9...
6,คมวิชช์ศัลยกรรมตกแต่ง,คลินิก,แขวงอรุณอมรินทร์ เขตบางกอกน้อย กรุงเทพมหานคร 1...,แพทย์ประจำคลินิกศัลยกรรมตกแต่ง โรงพยาบาลบำรุงร...,,https://www.yellowpages.co.th/profile/%E0%B8%8...
7,คลินิกศัลยกรรมตกแต่งเสริมสวย รังสิต,คลินิก,ตำบลประชาธิปัตย์ อำเภอธัญบุรี จังหวัดปทุมธานี ...,"ศัลยกรรมตกแต่งเสริมสวย,คลินิกศัลยกรรมตกแต่งเสร...",,https://www.yellowpages.co.th/profile/%E0%B8%8...
